In [ ]:
# Install the missing package
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [ ]:
# Install the Groq integration package
!pip install langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
# 📦 Install required packages
!pip install groq langchain faiss-cpu gradio tiktoken sentence-transformers

# 📚 Imports
import os, zipfile, tempfile, gradio as gr
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
# Update the import path for ChatGroq
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

# 🔐 Set your Groq API Key
os.environ["GROQ_API_KEY"] = "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"  # 🛑 Replace with your actual Groq key

# ✅ Initialize LLaMA 3 from Groq
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="llama3-8b-8192")

# ✅ Use HuggingFace Embeddings (no OpenAI needed)
embeddings = HuggingFaceEmbeddings()

# 📂 Allowed file extensions
EXTENSIONS = [".py", ".js", ".java", ".ts", ".html", ".txt", ".md"]

# 🔍 Load and chunk documents from ZIP
def extract_code_docs(zip_file):
    documents = []
    with tempfile.TemporaryDirectory() as tmpdir:
        with zipfile.ZipFile(zip_file.name, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)

        for root, _, files in os.walk(tmpdir):
            for fname in files:
                if any(fname.endswith(ext) for ext in EXTENSIONS):
                    path = os.path.join(root, fname)
                    try:
                        with open(path, "r", encoding="utf-8", errors="ignore") as f:
                            content = f.read()
                            documents.append(Document(page_content=content, metadata={"source": fname}))
                    except Exception as e:
                        print(f"⚠️ Skipped {fname}: {e}")
    return documents

# 🧠 Prepare QA Chain
def prepare_chain(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(docs, embeddings)
    retriever = vectorstore.as_retriever()
    return RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# 🔁 Gradio function
def ask_about_repo(zip_file, question):
    if zip_file is None:
        return "❗ Please upload a .zip file containing code."

    try:
        docs = extract_code_docs(zip_file)
        if not docs:
            return "⚠️ No valid code files found in the ZIP."

        qa_chain = prepare_chain(docs)
        response = qa_chain.run(question)
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}"

# 🎨 Gradio UI
gr.Interface(
    fn=ask_about_repo,
    inputs=[
        gr.File(label="📁 Upload GitHub ZIP Repo", file_types=[".zip"]),
        gr.Textbox(label="❓ Ask your question", lines=2)
    ],
    outputs=gr.Textbox(label="💡 Answer from Codebase"),
    title="🧠 Codebase QA Agent (Groq + LLaMA 3 + HuggingFace)",
    description="Upload a .zip of a code repo and ask technical questions like 'What does app.py do?' or 'Suggest code improvements.'"
).launch()


<ipython-input-12-1516050710>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-12-1516050710>:21: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingf

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://321ad7a1f05eb8ac48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
